In [ ]:
%load_ext autoreload

%autoreload 2

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

# Playing Field Mask Inspection

Visualize the HSV-based green field mask and explore how it filters player detections. Use this notebook to tune `hsv_lower`, `hsv_upper`, `morph_kernel_size`, and `min_overlap` so the mask reliably covers the pitch while rejecting crowd and off-field detections.

## Imports

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pyrootutils
from matplotlib.patches import Rectangle

from football_tracking_demo.config import load_config
from football_tracking_demo.detector import PlayerDetector
from football_tracking_demo.filtering import (
    build_playing_field_mask,
    filter_by_field_overlap,
)
from football_tracking_demo.video_io import get_video_metadata

## Parameters

In [ ]:
root = pyrootutils.setup_root(
    search_from=".",
    indicator="pyproject.toml",
    project_root_env_var=True,
    dotenv=True,
    pythonpath=True,
    cwd=True,
)

VIDEO_PATH = str(root / "data" / "match.mp4")
CONFIG_PATH = str(root / "config" / "config.yaml")

# Frames to sample (indices into the video)
SAMPLE_FRAME_INDICES = [0, 500, 1500, 3000]

# HSV mask parameters — defaults pulled from config playing_field_mask section.
# Edit here to explore different values without touching config.yaml.
config = load_config(CONFIG_PATH)
_mask_cfg = config.get("playing_field_mask", {})

HSV_LOWER = tuple(_mask_cfg.get("hsv_lower", [35, 40, 40]))
HSV_UPPER = tuple(_mask_cfg.get("hsv_upper", [85, 255, 255]))
MORPH_KERNEL_SIZE = _mask_cfg.get("morph_kernel_size", 15)
MIN_OVERLAP = _mask_cfg.get("min_overlap", 0.3)

print(
    f"HSV_LOWER={HSV_LOWER}  HSV_UPPER={HSV_UPPER}  kernel={MORPH_KERNEL_SIZE}  min_overlap={MIN_OVERLAP}"
)

## Load Video Metadata

Check basic video properties and confirm the file is accessible.

In [ ]:
meta = get_video_metadata(VIDEO_PATH)
meta

## Extract Sample Frames

Pull a handful of frames at different moments in the clip.

In [ ]:
cap = cv2.VideoCapture(VIDEO_PATH)
frames = {}
target_indices = set(SAMPLE_FRAME_INDICES)

for i in range(max(SAMPLE_FRAME_INDICES) + 1):
    ret, frame = cap.read()
    if not ret:
        break
    if i in target_indices:
        frames[i] = frame

cap.release()
print(f"Loaded {len(frames)} sample frames: {sorted(frames.keys())}")

## Raw Field Mask Visualization

Show the binary HSV mask alongside the original frame. White pixels indicate regions classified as the green playing field. The mask is built by:
1. Converting the BGR frame to HSV colour space.
2. Thresholding for green hues (`HSV_LOWER` → `HSV_UPPER`).
3. Applying morphological closing to fill holes and smooth the mask.

In [ ]:
def show_mask_overlay(frame_bgr, hsv_lower, hsv_upper, kernel_size, title_prefix=""):
    """Display frame, binary mask, and a green-tinted overlay side by side."""
    mask = build_playing_field_mask(frame_bgr, hsv_lower, hsv_upper, kernel_size)
    rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)

    # Tinted overlay: green where mask is active
    overlay = rgb.copy()
    green_region = mask > 0
    overlay[green_region] = (
        overlay[green_region] * 0.5 + np.array([0, 180, 0]) * 0.5
    ).astype(np.uint8)

    coverage = green_region.mean() * 100

    fig, axes = plt.subplots(1, 3, figsize=(20, 6))
    axes[0].imshow(rgb)
    axes[0].set_title("Original Frame", fontsize=12)
    axes[0].set_axis_off()

    axes[1].imshow(mask, cmap="gray")
    axes[1].set_title(f"Field Mask  ({coverage:.1f}% coverage)", fontsize=12)
    axes[1].set_axis_off()

    axes[2].imshow(overlay)
    axes[2].set_title("Mask Overlay (green = field)", fontsize=12)
    axes[2].set_axis_off()

    plt.suptitle(
        f"{title_prefix}  |  HSV [{hsv_lower}–{hsv_upper}]  kernel={kernel_size}",
        fontsize=13,
    )
    plt.tight_layout()
    plt.show()


for idx, frame in sorted(frames.items()):
    show_mask_overlay(
        frame,
        HSV_LOWER,
        HSV_UPPER,
        MORPH_KERNEL_SIZE,
        title_prefix=f"Frame {idx}",
    )

## HSV Parameter Sweep

Try different hue and saturation bounds to see their effect on field coverage. Wider hue range captures more of the pitch but risks including non-field green areas (e.g. advertising boards). Increasing saturation minimum (`s_min`) helps exclude washed-out or pale regions.

In [ ]:
sweep_frame = frames[SAMPLE_FRAME_INDICES[-1]]
sweep_frame = frames[SAMPLE_FRAME_INDICES[-2]]

hsv_candidates = [
    # (lower, upper, label)
    ((30, 30, 30), (90, 255, 255), "wide hue, low sat"),
    ((35, 40, 40), (85, 255, 255), "default"),
    ((38, 60, 60), (82, 255, 255), "narrow hue, med sat"),
    ((40, 80, 60), (80, 255, 255), "tight, high sat"),
]

rgb = cv2.cvtColor(sweep_frame, cv2.COLOR_BGR2RGB)
fig, axes = plt.subplots(2, len(hsv_candidates), figsize=(6 * len(hsv_candidates), 10))

for col, (lower, upper, label) in enumerate(hsv_candidates):
    mask = build_playing_field_mask(sweep_frame, lower, upper, MORPH_KERNEL_SIZE)
    overlay = rgb.copy()
    overlay[mask > 0] = (overlay[mask > 0] * 0.5 + np.array([0, 180, 0]) * 0.5).astype(
        np.uint8
    )
    coverage = (mask > 0).mean() * 100

    _ = axes[0, col].imshow(mask, cmap="gray")
    _ = axes[0, col].set_title(f"{label}\n{coverage:.1f}% coverage", fontsize=10)
    _ = axes[0, col].set_axis_off()

    _ = axes[1, col].imshow(overlay)
    _ = axes[1, col].set_axis_off()
    _ = axes[1, col].set_title(f"H:[{lower[0]},{upper[0]}]  S≥{lower[1]}", fontsize=10)

_ = plt.suptitle("HSV Parameter Sweep — Effect on Field Mask", fontsize=14)
plt.tight_layout()
plt.show()

## Morphological Kernel Size Effect

Morphological closing (`cv2.MORPH_CLOSE`) fills small holes in the mask. A larger kernel closes bigger gaps but also expands the mask outward. Compare kernel sizes to find a value that fills gaps between players without bleeding onto the stands.

In [ ]:
kernel_sizes = [5, 10, 25, 40, 60]

rgb = cv2.cvtColor(sweep_frame, cv2.COLOR_BGR2RGB)
fig, axes = plt.subplots(1, len(kernel_sizes), figsize=(5 * len(kernel_sizes), 5))

for ax, k in zip(axes, kernel_sizes):
    mask = build_playing_field_mask(sweep_frame, HSV_LOWER, HSV_UPPER, k)
    coverage = (mask > 0).mean() * 100
    _ = ax.imshow(mask, cmap="gray")
    _ = ax.set_title(f"kernel={k}\n{coverage:.1f}%", fontsize=11)
    _ = ax.set_axis_off()

_ = plt.suptitle(
    f"Morphological Closing — Kernel Size Comparison\nHSV {HSV_LOWER} → {HSV_UPPER}",
    fontsize=13,
)
plt.tight_layout()
plt.show()

## Detection Filtering: Before vs After

Run the YOLO detector and then apply the field overlap filter. Detections are shown in two colours:
- **Green** — kept (bottom half sufficiently overlaps the field mask)
- **Red** — rejected (too little field coverage in bottom half)

In [ ]:
det_cfg = config["detection"]

detector = PlayerDetector(
    model_name=det_cfg["model"],
    conf_threshold=det_cfg["confidence_threshold"],
    iou_threshold=det_cfg["nms_iou_threshold"],
    device=det_cfg["device"],
    hud_top=config["hud_mask"]["top_percent"],
    hud_bottom=config["hud_mask"]["bottom_percent"],
    hud_enabled=config["hud_mask"]["enabled"],
    shape_filter_config=config.get("detection_shape_filter"),
    # field_mask_config intentionally omitted here: this notebook manually
    # controls the mask via HSV_LOWER / HSV_UPPER / MORPH_KERNEL_SIZE above.
)

In [ ]:
def draw_kept_rejected(frame_bgr, kept, rejected, field_mask, title=""):
    """Overlay field mask + colour-coded bounding boxes."""
    rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
    overlay = rgb.copy()
    overlay[field_mask > 0] = (
        overlay[field_mask > 0] * 0.6 + np.array([0, 180, 0]) * 0.4
    ).astype(np.uint8)

    fig, ax = plt.subplots(1, 1, figsize=(15, 8))
    ax.imshow(overlay)

    for det in rejected:
        x1, y1, x2, y2 = det[:4]
        y_mid = (y1 + y2) / 2
        _ = ax.add_patch(
            Rectangle(
                (x1, y1),
                x2 - x1,
                y2 - y1,
                linewidth=2,
                edgecolor="red",
                facecolor="none",
            )
        )
        _ = ax.add_patch(
            Rectangle(
                (x1, y_mid),
                x2 - x1,
                y2 - y_mid,
                linewidth=0,
                facecolor="red",
                alpha=0.25,
            )
        )

    for det in kept:
        x1, y1, x2, y2 = det[:4]
        y_mid = (y1 + y2) / 2
        _ = ax.add_patch(
            Rectangle(
                (x1, y1),
                x2 - x1,
                y2 - y1,
                linewidth=2,
                edgecolor="lime",
                facecolor="none",
            )
        )
        _ = ax.add_patch(
            Rectangle(
                (x1, y_mid),
                x2 - x1,
                y2 - y_mid,
                linewidth=0,
                facecolor="lime",
                alpha=0.20,
            )
        )

    ax.set_title(
        f"{title}  |  kept={len(kept)}  rejected={len(rejected)}  "
        f"(min_overlap={MIN_OVERLAP})",
        fontsize=12,
    )
    ax.set_axis_off()
    # Legend
    ax.add_patch(
        Rectangle(
            (0, 0), 0, 0, edgecolor="lime", facecolor="none", linewidth=2, label="kept"
        )
    )
    ax.add_patch(
        Rectangle(
            (0, 0),
            0,
            0,
            edgecolor="red",
            facecolor="none",
            linewidth=2,
            label="rejected",
        )
    )
    ax.legend(loc="upper right", fontsize=10, framealpha=0.8)
    plt.tight_layout()
    plt.show()


for idx, frame in sorted(frames.items()):
    detections = detector.detect(frame)
    field_mask = build_playing_field_mask(
        frame, HSV_LOWER, HSV_UPPER, MORPH_KERNEL_SIZE
    )
    kept = filter_by_field_overlap(detections, field_mask, MIN_OVERLAP)
    kept_set = {tuple(d[:4]) for d in kept}
    rejected = [d for d in detections if tuple(d[:4]) not in kept_set]

    draw_kept_rejected(frame, kept, rejected, field_mask, title=f"Frame {idx}")

## Overlap Threshold Sweep

The `min_overlap` parameter controls how much of a detection's bottom half must lie on the field for it to be kept. Lower values are more permissive (keep more detections); higher values are stricter (keep only detections clearly on the pitch).

In [ ]:
sweep_frame_idx = SAMPLE_FRAME_INDICES[1]
sweep_frame = frames[sweep_frame_idx]
all_detections = detector.detect(sweep_frame)
field_mask = build_playing_field_mask(
    sweep_frame, HSV_LOWER, HSV_UPPER, MORPH_KERNEL_SIZE
)

overlap_thresholds = [0.0, 0.1, 0.2, 0.3, 0.5, 0.7]
rgb = cv2.cvtColor(sweep_frame, cv2.COLOR_BGR2RGB)

fig, axes = plt.subplots(2, 3, figsize=(21, 12))

for ax, thresh in zip(axes.flat, overlap_thresholds):
    kept = filter_by_field_overlap(all_detections, field_mask, thresh)
    kept_set = {tuple(d[:4]) for d in kept}
    rejected = [d for d in all_detections if tuple(d[:4]) not in kept_set]

    overlay = rgb.copy()
    overlay[field_mask > 0] = (
        overlay[field_mask > 0] * 0.7 + np.array([0, 180, 0]) * 0.3
    ).astype(np.uint8)

    _ = ax.imshow(overlay)
    for det in rejected:
        x1, y1, x2, y2 = det[:4]
        _ = ax.add_patch(
            Rectangle(
                (x1, y1),
                x2 - x1,
                y2 - y1,
                linewidth=1.5,
                edgecolor="red",
                facecolor="none",
            )
        )
    for det in kept:
        x1, y1, x2, y2 = det[:4]
        _ = ax.add_patch(
            Rectangle(
                (x1, y1),
                x2 - x1,
                y2 - y1,
                linewidth=1.5,
                edgecolor="lime",
                facecolor="none",
            )
        )

    _ = ax.set_title(
        f"min_overlap={thresh:.1f}  →  kept {len(kept)}/{len(all_detections)}",
        fontsize=11,
    )
    _ = ax.set_axis_off()

_ = plt.suptitle(
    f"Overlap Threshold Sweep — Frame {sweep_frame_idx}  (green=kept, red=rejected)",
    fontsize=14,
)
plt.tight_layout()
plt.show()

## Summary: Kept Detections vs Threshold

Plot the number of kept detections as a function of `min_overlap` across all sample frames to understand the sensitivity of the filter.

In [ ]:
thresholds = np.linspace(0.0, 1.0, 21)

fig, ax = plt.subplots(figsize=(10, 5))

for idx, frame in sorted(frames.items()):
    dets = detector.detect(frame)
    fmask = build_playing_field_mask(frame, HSV_LOWER, HSV_UPPER, MORPH_KERNEL_SIZE)
    kept_counts = [len(filter_by_field_overlap(dets, fmask, t)) for t in thresholds]
    _ = ax.plot(thresholds, kept_counts, marker="o", markersize=4, label=f"frame {idx}")

_ = ax.axvline(
    x=MIN_OVERLAP,
    color="gray",
    linestyle="--",
    label=f"current min_overlap={MIN_OVERLAP}",
)
_ = ax.set_xlabel("min_overlap threshold", fontsize=12)
_ = ax.set_ylabel("detections kept", fontsize=12)
_ = ax.set_title("Field Overlap Filter: Kept Detections vs Threshold", fontsize=13)
_ = ax.legend(fontsize=10)
_ = ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()